In [39]:
import xarray as xr
import os
from zarr.storage import DirectoryStore
import s3fs
import shutil
import tempfile
import fsspec
import numpy as np
import zarr
from numpy.core import product

In [40]:
np.__version__

'1.26.0'

In [41]:
np.__file__

'/home/administrator-se/shree_zarr/lib/python3.12/site-packages/numpy/__init__.py'

In [42]:
xr.__version__

'2023.1.0'

In [43]:
zarr.__version__

'2.13.3'

In [44]:

s3 = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)
bucket_name = 'transition-plan-data'
prefix = 'wind-speed-direction-latlon-data/2024'

In [45]:
 
# Load the netCDF file

wind_speed_nc = '/media/administrator-se/HARDDISK20TB/se_drive1/wind-model/nc-files/100m-wind/2024/uv/wind_speed.nc'

wind_direction_nc = '/media/administrator-se/HARDDISK20TB/se_drive1/wind-model/nc-files/100m-wind/2024/uv/wind_direction.nc'

speed_ds= xr.open_dataset(wind_speed_nc)

direction_ds= xr.open_dataset(wind_direction_nc)

speed_ds = speed_ds.rename({'valid_time':'time'})
direction_ds = direction_ds.rename({'valid_time':'time'})

# Extract the wind_direction variable

wind_speed = speed_ds['wind_speed']

wind_direction = direction_ds['wind_direction']

In [46]:
def process_all_grid_points(wind_speed,wind_direction,prefix,bucket_name):
    
    time_dim, lat_dim, lon_dim = wind_direction.shape
    # Checkpoint file to keep track of progress
    checkpoint_file = 'checkpoint.txt'
    
    # Load checkpoint if it exists
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            start_lat = int(f.read().strip())
    else:
        start_lat = 0  # Start from the beginning if no checkpoint
    
    try:
        for lat in range(start_lat, lat_dim):
            for lon in range(lon_dim):
                print(f'{lat}_{lon}')
                # Extract the time series for the current grid point
                wind_speed_latlon = wind_speed[:, lat, lon]
                wind_direction_latlon = wind_direction[:, lat, lon]
    
                combined_ds = xr.Dataset({
                    'wind_direction': wind_direction_latlon,
                    'wind_speed': wind_speed_latlon
                })
    
                # Define the output zarr file name
                zarr_local = f'{lat}_{lon}.zarr'

                # Save the combined time series as a zarr file
                # combined_ds.to_zarr(zarr_local, mode='w', consolidated = False)
                
                combined_ds.to_zarr(store = zarr.DirectoryStore(zarr_local), mode='w', consolidated = False)
                
                zarr_filename = f'{prefix}/{lat}_{lon}.zarr'
                s3_path = f'{bucket_name}/{zarr_filename}'
    
                # Upload the local zarr file to S3
                s3.put(zarr_local, s3_path, recursive=True)
                #print('uploaded')
    
                # Remove local zarr file to save space
                shutil.rmtree(zarr_local)
    
            # Update checkpoint file
            with open(checkpoint_file, 'w') as f:
                f.write(str(lat))
    
            print(f'Completed {lat} latitude.')
    
    
        print("All grid point time series have been saved as zarr files.")
    
    except Exception as e:
        print(f"An error occurred: {e}")


In [47]:
def run_code():
    while True:
        try:
            process_all_grid_points(wind_speed,wind_direction,prefix=prefix, bucket_name=bucket_name)
            break  # Break out of the loop if the code runs successfully
        except Exception as e:
            print("An error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)  # Wait for 5 seconds before retrying

# Call the function to run the code
run_code()

0_0
0_1
0_2
0_3
0_4
0_5
0_6
0_7
0_8
0_9
0_10
0_11
0_12


KeyboardInterrupt: 

In [ ]:
def run_code():
    while True:
        try:
            process_all_grid_points(wind_speed,wind_direction,prefix=prefix, bucket_name=bucket_name)
            break  # Break out of the loop if the code runs successfully
        except Exception as e:
            print("An error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)  # Wait for 5 seconds before retrying

# Call the function to run the code
run_code()